In [0]:
%r
# function to bin data and calculate mean prediction/y values
bin_data <- function(df, n_bins, strat_col=NULL) {
      
  if (is.null(strat_col)) {
    # no stratification
    binned_data <- mutate(df, bin=ntile(get("prediction"), n_bins)) %>%
      group_by(bin) %>%
      mutate(n=n(),
             bin_pred=mean(get("prediction")),
             bin_prob=mean(as.numeric(y)),
             se=sqrt((bin_prob * (1-bin_prob)) / n),
             ul=bin_prob + 1.96*se,
             ll=bin_prob - 1.96*se)
  } else {
    binned_data <- df %>%
       group_by(!!strat_col) %>%
       mutate(bin=ntile(get("prediction"), n_bins)) %>%
       group_by(bin, !!strat_col) %>%
       mutate(n=n(),
              bin_pred=mean(get("prediction")),
              bin_prob=mean(as.numeric(y)),
              se=sqrt((bin_prob * (1-bin_prob)) / n),
              ul=bin_prob + 1.96*se,
              ll=bin_prob - 1.96*se)
    }
  return(binned_data)
}

In [0]:
%r
# generate binned calibration plot
plot_overall_calibration <- function(df, n_bins, title=NULL) {
  # calibration plot containing bined data
  binned_data <- bin_data(df, n_bins=n_bins)
    
  p <- ggplot(binned_data, aes(x=bin_pred, y=bin_prob, ymin=ll, ymax=ul)) +
          geom_abline() +
          geom_pointrange(size=0.4, color=ronin_colors[1]) +
          coord_cartesian(xlim=c(0,0.4), ylim=c(0,0.4)) +
          geom_smooth(method = "lm", se = FALSE, formula = y~-1 + x, aes(color="Linear Fit", linetype="Linear Fit")) +
          geom_smooth(aes(x = get("prediction"), y = as.numeric(y), color="Loess Fit", linetype="Loess Fit"), se = FALSE, method = "loess") +
          scale_color_manual(name="legend", values=ronin_colors) +
          scale_linetype_manual(name="legend", values=c("dashed", "solid")) +
          xlab("Predicted Probability") +
          ylab("Observed Probability") +
          ggtitle(title) +
          theme_minimal() +
          theme(legend.position=c(0.7, 0.3),
                legend.title=element_blank(),
                panel.grid.minor = element_blank())
  
  return(p)
}

In [0]:
%r
# function to calculate CI from bootstraps
calc_ci <- function(mrns, preds, obs, fun, n_boot=100) {
  
  
  df <- data.frame(mrns=mrns, preds=preds, obs=obs)
  boot_output <- rep(NA, n_boot)
  
  for (i in 1:n_boot) {
    # randomly sample MRNs with replacement
    unique_mrns <- unique(df$mrns)
    rand_mrns <- sample_n(as.data.frame(unique_mrns), length(unique_mrns), replace=TRUE)
    samp_df <- df[df$mrns %in% rand_mrns$unique_mrns,]
    
    #samp_df <- sample_n(df, nrow(df), replace=TRUE)
    boot_output[i] <- fun(samp_df$preds, samp_df$obs)
  }
  
  quants <- quantile(boot_output, c(0.025, 0.5, 0.975))
  lower <- quants[1]
  upper <- quants[3]
  med <- quants[2]
  
  return(c(lower, upper, med))
}

In [0]:
%r
#' Calculate the integrated calibration index (ICI) for predicted probabilities against a binary outcome. Based on Austin PC, Steyerberg EW. The Integrated Calibration Index (ICI) and related metrics for quantifying the calibration of logistic regression models. Statistics in Medicine. 2019;1–15. https://doi.org/10.1002/sim.8281
# See appendix from Austin and Steyerberg


cal_slope <- function(preds, obs) {
  fit <- glm(obs ~ preds, data=week_df)
  slope <- fit$coefficients["preds"]
  return(slope)
}

cal_intercept <- function(preds, obs) {
  fit <- glm(obs ~ preds, data=week_df)
  intercept <- fit$coefficients["(Intercept)"]
  return(intercept)
}

ici <- function(preds, obs) {
  loess.calibrate <- loess(obs ~ preds)
  p.calibrate <- predict(loess.calibrate, newdata=preds)
  ici <- mean(abs(p.calibrate - preds))
  return(ici)
}

e50 <- function(preds, obs) {
  loess.calibrate <- loess(obs ~ preds)
  p.calibrate <- predict(loess.calibrate, newdata=preds)
  e50 <- median(abs(p.calibrate - preds))
  return(e50)
}

e90 <- function(preds, obs) {
  loess.calibrate <- loess(obs ~ preds)
  p.calibrate <- predict(loess.calibrate, newdata=preds)
  e90 <- quantile(abs(p.calibrate - preds), probs=0.9)
  return(e90)
}

emax <- function(preds, obs) {
  loess.calibrate <- loess(obs ~ preds)
  p.calibrate <- predict(loess.calibrate, newdata=preds)
  emax <- max(abs(p.calibrate - preds))
  return(emax)
}

brier <- function(preds, obs) {
  brier <- mean((obs - preds)^2)
  return(brier)
}

# https://gweissman.github.io/post/evaluating-the-equivalence-of-different-formulations-of-the-scaled-brier-score/
brier_scaled <- function(preds, obs) {
  brier <- brier(preds, obs)
  brier_max <- mean((obs - mean(obs))^2)
  brier_scaled <- 1 - brier/brier_max
  return(brier_scaled)
}